# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset

ds = load_dataset("CUTD/news_articles_df")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


news_articles_df.csv:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8378 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'summarizer', 'text'],
        num_rows: 8378
    })
})

In [4]:
ds = ds['train']

## remove columns

In [5]:
ds = ds.remove_columns(['Unnamed: 0'])

In [6]:
ds

Dataset({
    features: ['summarizer', 'text'],
    num_rows: 8378
})

## split data

In [7]:
ds = ds.train_test_split(test_size=0.2)

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [8]:
from transformers import AutoTokenizer

checkpoint = "yalsaffar/mt5-small-Arabic-Summarization"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [9]:
def PreprocessFunction(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summarizer"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_ds = ds.map(PreprocessFunction, batched=True)

Map:   0%|          | 0/6702 [00:00<?, ? examples/s]

Map:   0%|          | 0/1676 [00:00<?, ? examples/s]

In [11]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['summarizer', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6702
    })
    test: Dataset({
        features: ['summarizer', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1676
    })
})

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [12]:
from transformers import DataCollatorForSeq2Seq

Collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="our text_Summarization",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
)

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=Collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,1.843600,1.464681


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#s

TrainOutput(global_step=1341, training_loss=1.9181557697293299, metrics={'train_runtime': 762.5877, 'train_samples_per_second': 8.788, 'train_steps_per_second': 1.758, 'total_flos': 2535176108206080.0, 'train_loss': 1.9181557697293299, 'epoch': 1.0})

## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [16]:
model.save_pretrained('./tex_summrize_model')
tokenizer.save_pretrained('./tex_summrize_model_tokenizer')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}


('./tex_summrize_model_tokenizer/tokenizer_config.json',
 './tex_summrize_model_tokenizer/special_tokens_map.json',
 './tex_summrize_model_tokenizer/spiece.model',
 './tex_summrize_model_tokenizer/added_tokens.json',
 './tex_summrize_model_tokenizer/tokenizer.json')

## Step 9: Inference

In [17]:
text="أطلقت الهيئة السعودية للذكاء الاصطناعي 'سدايا' الهوية الجديدة للقمة العالمية للذكاء الاصطناعي بنسختها الثالثة تحت شعار لخير البشرية التي استوحتها من رمزية تقنيات الذكاء الاصطناعي، وجرى بلورتها في أربعة حروف هي : (GAIN) اختصارًا للعبارة الإنجليزية GLOBAL AI SUMMIT.وتهدف حروف (GAIN) إلى الفلسفة العميقة لمفهوم الذكاء الاصطناعي حيث يرمز حرف G إلى التوسع والتطور الرقمي العالمي، بينما يرمز الحرفان AI لمصطلح الذكاء الاصطناعي، ويأتي حرف N اختصارًا لعبارات NOW-NEXT - NEVER وهي عبارة عن ترسيخ مفهوم البيانات والذكاء الاصطناعي بأنها للحاضر وللمستقبل الواعد.وتتكون حروف (GAIN) من مزيج من ألوان الأبيض والبنفسجي والأسود، ويرمز اللون البنفسجي إلى دلالات الابتكار والشغف والتطور التقني، واللون الأسود إلى سطوة تقنيات البيانات والذكاء الاصطناعي وقوة حضورها في عصرنا الحالي في حين يرمز اللون الأبيض إلى النقاء."

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [18]:
from transformers import pipeline

SummarizerModel = pipeline("summarization", model="/content/tex_summrize_model", tokenizer="/content/tex_summrize_model_tokenizer")
SummarizerModel(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': 'أطلقت الهيئة السعودية للذكاء الاصطناعي بنسختها الثالثة تحت شعار لخير البشرية.'}]